#### Use Bloom 3b for Q&A along with SetFit few shots examples and examine if it improves Q&A over baseline

##### Setup

In [ ]:
# Bloom is part of the transformers library --> install it
!pip install --quiet transformers

     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 74.6 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 


In [ ]:
# imports needed libraries
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM # A general model for casual inferencing

In [ ]:
# test GPU avialiablity - otherwise cpu

!nvidia-smi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



device(type='cpu')

In [ ]:
# if we have gpu bind all tensors to gpu, otherwise by default cpu

if 'cuda' in str(device):
  torch.set_default_tensor_type(torch.cuda.FloatTensor) # this will allocate all tensors  on cuda



#### Using bloom 3b for text prediction 

In [ ]:
# define the tokenizer and model 

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b") # here we use bloom 3b parameters

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

In [ ]:
# define any text prompt / or a list of prompts 
# in this example we have text continuation task as well as general knowledge question/answering task mixed together
text_prompt = ["Albert Einstein won a Nobel prize for", "Otto Warburg won discovered" ]# ANyone can change this part

In [ ]:
# let's look at how bloom tokenizes text
tokens = tokenizer.tokenize(text_prompt)
print(tokens)

['Albert', 'ĠEinstein', 'Ġwon', 'Ġa', 'ĠNobel', 'Ġprize', 'Ġfor', 'Ot', 'to', 'ĠWar', 'burg', 'Ġwon', 'Ġdiscovered']


In [ ]:
# convert tokens to inpt ids and return pytorch tensors
input_ids = tokenizer (text_prompt, return_tensors="pt", padding=True)
input_ids

{'input_ids': tensor([[124190,  78426,  15974,    267,  41530, 127901,    613],
        [     3, 125112,   1025,  18692,  19616,  15974,  54419]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# generate tokens as ids and review the generated ids
gen_text = model.generate(**input_ids, min_length=20, max_length=40, temperature=0.2)

In [ ]:
gen_text

tensor([[124190,  78426,  15974,    267,  41530, 127901,    613,   3868,   2909,
            664,    368,  25044,    461,  18074,   1185,     17,   5298,   1620,
            368,   3968,   2298,    427,   2971,    368,  14679,   1503,   8802,
         144516,    982,    361,   3868,  41530,  43972,     17,   5298,   7555,
             15,   1503,   7994,   1306],
        [     3, 125112,   1025,  18692,  19616,  15974,  54419,    861,    368,
          16659,    461,  58331,  62079,    361,    368, 143312,   2846,    461,
          26073,  13953,   1620,   4936,  17952,   4340,    861,    461,   8342,
          13953,     17,   3904,  80434,  27531,    427,    368,  11468,    461,
            368,  18692,  19616,   6416]])

In [ ]:
# predict possible continuation for the provided prompts

predictions = []
for index, _ in enumerate(gen_text):
  predicted_text = tokenizer.decode(gen_text[index])
  # print(f"Paragraph {index} is: {predicted_text}\n")
  predictions.append(predicted_text)

In [ ]:
# do some cleaning of text:
# 1. find the last "." and delete everything afterwords to have a clean sentense
# 2. remove the new line character \n to make it easier and review
# 3. remove <pad> tokens

for item, pred in enumerate(predictions):
  pred = pred[0:pred.rfind(".")+1] # truncate words beyond last period
  pred = pred.replace("\n", "") # remove newlines
  pred = pred.replace("<pad>", "") # remove padding tokens
  print(f"Paragraph {item}: {pred}\n")

Paragraph 0: Albert Einstein won a Nobel prize for his work on the theory of relativity. He was the first person to use the word “relativity” in his Nobel lecture.

Paragraph 1: Otto Warburg won discovered that the rate of oxygen consumption in the mitochondria of cancer cells was much higher than that of normal cells.



#### Using bloom 3b parameters for zero-shot Q&A

In [ ]:
# define the tokenizer and model 

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b") # here we use bloom 3b parameters

In [ ]:
# text with several general knowledge questions

prompt = ["Who was Otto Warburg and what he is known for?", "What Prof. Thomas Seyfried theory about cancer as a metabolic disease says?",\
          "What is the role of the m-Tor pathway in cancer progression?", \
          "What are the best natural blood Glucose inhibitors?",
          "Is the theory of cancer as a metabolic disease correct or not?"]

In [ ]:
# convert it into input_ids
input_ids = tokenizer(prompt, return_tensors='pt', padding=True)

In [ ]:
# generate predicted ids that should now be answers
generated_ids = model.generate(**input_ids, min_length=20, max_length=80, temperature=0.2, repetition_penalty=1.1)

In [ ]:
# transform it back to text

predictions = []
for ids in generated_ids:
  predicted_text = tokenizer.decode(ids)
  print(f"Q&A: {predicted_text}\n")
  predictions.append(predicted_text)

Q&A: <pad><pad><pad>Who was Otto Warburg and what he is known for? The German scientist who discovered the significance of glucose metabolism in cancer cells. He also developed a method to measure oxygen consumption by living organisms.
The first thing you should know about Otto Warburg is that he was born on July 25, 1875 in Vienna, Austria. His father was a physician and his mother was an artist. He

Q&A: What Prof. Thomas Seyfried theory about cancer as a metabolic disease says? What is the difference between metabolism and energy production?
The most important thing to know about metabolism is that it is an essential process in all living organisms, including humans.
Metabolism is the chemical reactions that take place inside cells to produce energy from food or other sources of fuel. Metabolism also helps us store nutrients for

Q&A: <pad>What is the role of the m-Tor pathway in cancer progression? The mTOR pathway has been implicated in a variety of cellular processes, including 

In [ ]:
# do some cleaning of text:
# 1. find the last "." and delete everything afterwords to have a clean sentense
# 2. remove the new line character \n to make it easier and review
# 3. remove <pad> tokens

for item, pred in enumerate(predictions):
  pred = pred[0:pred.rfind(".")+1] # truncate words beyond last period
  pred = pred.replace("\n", "") # remove newlines
  pred = pred.replace("<pad>", "") # remove padding tokens
  print(f"Paragraph {item}: {pred}\n")

Paragraph 0: Who was Otto Warburg and what he is known for? The German scientist who discovered the significance of glucose metabolism in cancer cells. He also developed a method to measure oxygen consumption by living organisms.The first thing you should know about Otto Warburg is that he was born on July 25, 1875 in Vienna, Austria. His father was a physician and his mother was an artist.

Paragraph 1: What Prof. Thomas Seyfried theory about cancer as a metabolic disease says? What is the difference between metabolism and energy production?The most important thing to know about metabolism is that it is an essential process in all living organisms, including humans.Metabolism is the chemical reactions that take place inside cells to produce energy from food or other sources of fuel.

Paragraph 2: What is the role of the m-Tor pathway in cancer progression? The mTOR pathway has been implicated in a variety of cellular processes, including cell growth and proliferation.

Paragraph 3: Wh

#### Using bloom 3b parameters for general Q&A using Q&A tokenizer and model and employing few shot examples

## Note: This does not work - it is not yet traind for Q&A. IN a seperate notebook we will train it for Q&A

In [ ]:
# define tokenizer and model and use one specific for question-answering

from transformers import AutoModelForQuestionAnswering, AutoModelForDocumentQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

model = AutoModelForQuestionAnswering.from_pretrained("bigscience/bloom-3b") # as we can see this does not work - it is nnot yet traind for Q&A

#### As we can see this does not work - it is nnot yet traind for Q&A. IN a seperate notebook we will train it for Q&A